In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from keras.utils import to_categorical
from keras.optimizers import SGD
from sklearn.utils import shuffle

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X = pd.read_csv('A_Z_HandwrittenData.csv', header=0)
X = shuffle(X)
X.columns = ['label'] + ['Pixel{}'.format(i) for i in range(784)]

X.head()

,label,Pixel0,Pixel1,Pixel2,Pixel3,Pixel4,Pixel5,Pixel6,Pixel7,Pixel8,...,Pixel774,Pixel775,Pixel776,Pixel777,Pixel778,Pixel779,Pixel780,Pixel781,Pixel782,Pixel783
125788,13,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
237603,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
214563,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
175724,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
154482,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
Y = to_categorical(X.label.values, num_classes=26)
X = X[['Pixel{}'.format(i) for i in range(784)]].values
X = X.reshape(-1, 28, 28, 1)

X_train = X[:300000]
X_test = X[300000:]
del X

Y_train = Y[:300000]
Y_test = Y[300000:]
del Y

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(300000, 28, 28, 1) (300000, 26) (72037, 28, 28, 1) (72037, 26)


In [6]:
from scipy.misc import imsave

X = X_train.reshape(-1, 28, 28)
for i in range(100):
    imsave('img/images_{}.jpg'.format(i), X[i, :, :])

del X

In [5]:
#The Model
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='SAME', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (5, 5), padding='SAME', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(26))
model.add(Activation(activation="softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               313700    
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [6]:
batch = 500
eps = 5

opt = SGD(lr=0.01, decay=1e-5)
model.compile(opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch, epochs=eps, validation_data=(X_test, Y_test))

Train on 300000 samples, validate on 72037 samples
Epoch 1/5
300000/300000 [==============================] - 907s 3ms/step - loss: 1.0120 - acc: 0.7633 - val_loss: 0.1755 - val_acc: 0.9535
Epoch 2/5
300000/300000 [==============================] - 871s 3ms/step - loss: 0.3192 - acc: 0.9108 - val_loss: 0.1273 - val_acc: 0.9667
Epoch 3/5
300000/300000 [==============================] - 885s 3ms/step - loss: 0.2431 - acc: 0.9333 - val_loss: 0.1090 - val_acc: 0.9721
Epoch 4/5
300000/300000 [==============================] - 977s 3ms/step - loss: 0.2054 - acc: 0.9437 - val_loss: 0.1016 - val_acc: 0.9736
Epoch 5/5
300000/300000 [==============================] - 859s 3ms/step - loss: 0.1808 - acc: 0.9514 - val_loss: 0.0903 - val_acc: 0.9764
